## 常用变量

### 波动率

In [1]:
# 计算原始价格序列波动率
def annual_std_volatility(series, windows_std):
    """计算原始序列的波动率
    Calculation of the volatility of the original sequence.
    Parameters
    ==========
    series: array
    windows_std: int
    Object to calculate the volatility
    """
    series = np.array(series)
    series_ema = series - talib.EMA(series, windows_std)
    series_sqt = series_ema ** 2
    std_series_ema = talib.EMA(series_sqt, windows_std)
    return np.sqrt(std_series_ema * 256.0 / windows_std)


# 计算价格序列收益率的波动率
def annual_std_return_volatility(series, windows_std):
    """计算给定序列收益率的波动率
    Calculation of the volatility of return.
    """
    series_return = (series[1:] / series[:-1]) - 1.0
    return annual_std_volatility(series_return, windows_std)

### 获取主力合约

In [2]:
def get_continuous_contract(api, product, trade_date):
    """给定连1、连2合约的代码：如 product = ["cuZ0.CF", "cuZ1.CF"]
    返回给定合约，当前时刻对应的连续合约代码
    trade_date: int,20150101
    """
    codes = [dataAPI.getContinuousSymbol(str(symbol), trade_date) for symbol in product]
    return codes


# 最好是按照下面的方式写，直接把合约作为局部变量，而不是作为参数传入
def get_contract(trade_date):
    """获取交易合约的第一种方式
    trade_date的输入格式如20150101
    以rb的换仓时间点为基准
    """
    symbols = ['rbZ0.CF', 'iZ0.CF', 'jZ0.CF']
    rb_symbol = dataAPI.getContinuousSymbol(symbols[0], trade_date)
    contract = slice(2, len(rb_symbol))
    codes1 = [symbol.replace('Z0.CF', rb_symbol[contract]) for symbol in symbols]
    codes2 = [symbol.replace('10', '09') for symbol in codes1]
    codes2[0] = rb_symbol
    return codes2

### 设计交易策略

In [4]:
# 按照设计函数的方式，返回trade_side列表，以后就可以直接调用
def strategy_boll_band_breakout(spread_df):
    """
    根据给定的spread_df序列(spread_df 为 pd.Series格式)，
    根据布林带动量突破逻辑，作为开平仓信号
    计算出每次开仓信号返回：trade_side列表
    """
    spread = spread_df.values
    trade_side = [0, 0]
    windows_len_mean = 20
    std_open = 1
    upper_band, middle, lower_band = talib.BBANDS(spread, timeperiod=windows_len_mean,
                                                  nbdevup=std_open, nbdevdn=std_open, matype=0)
    # 开仓条件
    if spread[-1] > upper_band[-1] and spread[-2] <= upper_band[-2]:
        trade_side.append(1)    # 突破上轨开多
    elif spread[-1] < lower_band[-1] and spread[-2] >= lower_band[-2]:
        trade_side.append(-1)   # 突破下轨开空

    # 平仓条件
    if (spread[-1] >= middle[-1] and spread[-2] <= middle[-2]) and (trade_side[-1] == -1):
        trade_side.append(0)  # 空平
    elif (spread[-1] <= middle[-1] and spread[-2] >= middle[-2]) and (trade_side[-1] == 1):
        trade_side.append(0)  # 多平

    return trade_side

### 获取收盘价（或者加入高开低收成交量）

In [5]:
def get_history_close(api, symbol, n_days):
    """
    获取给定 symbol 过去 n_days个交易日的数据
    少于 n_days就以上一年度的数据填充。
    eg:取'j1501'过去120个交易日数据
       少于部分以'j1401'填充
    数据返回 DataFrame格式
    """
    year = slice(-7, -3)
    symbol_last = symbol[:-7] + str(int(symbol[year]) - 100) + symbol[-3:]
    str_fields = ['tradeDate', 'close']
    str_index = 'tradeDate'
    df_close = api.getBarsHistory(symbol=symbol, timeSpan=ETimeSpan.DAY_1, skipSuspended=1, count=n_days,
                                  df=True, priceMode=EPriceMode.REAL, fields=str_fields).set_index(str_index)
    length = len(df_close)
    if length < n_days:
        n_days = n_days - length
        df_last = api.getBarsHistory(symbol=symbol_last, timeSpan=ETimeSpan.DAY_1, skipSuspended=1, count=n_days,
                                     df=True, priceMode=EPriceMode.REAL, fields=str_fields).set_index(str_index)
        df = pd.concat([df_last, df_close])
    else:
        df = df_close
    return df

### 计算交易手数

In [6]:
def capital_lots(used_capital_ratio, close, capital, multiple, leverage, cumpnl_raw):
    """根据资金使用比例，计算开仓手数
    return
    ======
    lots: 开仓手数
    """
    margin_rate = 0.2  # 保证金比例， 10%
    can_used_market_value = cumpnl_raw / margin_rate
    limit_lots = int(can_used_market_value / (2 * close * multiple))
    # 资金设置
    lots = int(used_capital_ratio * leverage * capital / (close * multiple))
    lots = np.min((lots, limit_lots))
    if lots == 0:
        lots = 1
    return lots

### 计算资金使用占比， 资金管理策略

In [7]:
def capital_used_ratio(api, account_equity):
    """
    根据账户权益，对策略整体收益率的波动率建模预测，计算资金使用率，用于仓位分配
    Parameters
    ==========
    account_equity: array like.
    返回：float, return the rate of use of capital
    """
    if len(account_equity) >= 120:
        std_return_volatility = annual_std_return_volatility(account_equity, 90)
        X = sm.add_constant(std_return_volatility[-41:-1], prepend=True)
        beta_hat = sm.RLM(std_return_volatility[-40:], X).fit().params
        std_return_volatility_predict = beta_hat[0] + beta_hat[1] * std_return_volatility[-1]
        used_capital_ratio = max(api.account_level_min,
                                 min(api.vol_target_S / std_return_volatility_predict, api.account_level_max))
    else:
        used_capital_ratio = 0.8
    return used_capital_ratio

### 计算合约价值与最小变动单位

In [ ]:
def get_multiplier(symbol):
    """
    获取给定symbol标的的基本信息
    Returns:
    =======
    marginRate:  保证金比例
    valuePerUnit: 合约价值
    priceTick: 最小变动单位
    只返回合约价值valuePerUnit
    """
    ref_data = dataAPI.getRefData(symbol)
    multiplier = ref_data.valuePerUnit
    priceTick = ref_data.priceTick
    return multiplier, priceTick